<a href="https://colab.research.google.com/github/DamienGulliver/Institute_of_data/blob/main/capstone_documentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of Drug Mechanism of Action

The Connectivity Map, a project within the Broad Institute of MIT and Harvard, the Laboratory for Innovation Science at Harvard (LISH), and the NIH Common Funds Library of Integrated Network-Based Cellular Signatures (LINCS), present this dataset with the goal of advancing drug development through improvements to MoA prediction algorithms.

## What is the Mechanism of Action (MoA) of a drug? And why is it important?

In the past, scientists derived drugs from natural products or were inspired by traditional remedies. Very common drugs, such as paracetamol, known in the US as acetaminophen, were put into clinical use decades before the biological mechanisms driving their pharmacological activities were understood. Today, with the advent of more powerful technologies, drug discovery has changed from the serendipitous approaches of the past to a more targeted model based on an understanding of the underlying biological mechanism of a disease. 

In this new framework, scientists seek to identify a protein target associated with a disease and develop a molecule that can modulate that protein target. As a shorthand to describe the biological activity of a given molecule, scientists assign a label referred to as mechanism-of-action or MoA for short.

##How do we determine the MoAs of a new drug?

One approach is to treat a sample of human cells with the drug and then analyze the cellular responses with algorithms that search for similarity to known patterns in large genomic databases, such as libraries of gene expression or cell viability patterns of drugs with known MoAs.

This project uses a unique dataset that combines gene expression and cell viability data. The data is based on a new technology that measures simultaneously (within the same samples) human cells’ responses to drugs in a pool of 100 different cell types (thus solving the problem of identifying ex-ante, which cell types are better suited for a given drug). In addition, the data contains MoA annotations for more than 5,000 drugs.  

The aim of the project is to develop an algorithm that automatically labels each case in the test set as one or more MoA classes. Note that since drugs can have multiple MoA annotations, the task is formally a multi-label classification problem.

## Dataset

 - train_features.csv - Features for the training, validation and test set. Features g- signify gene expression data, and c- signify cell viability data. cp_type indicates samples treated with a compound (cp_vehicle) or with a control perturbation (ctrl_vehicle); control perturbations have no MoAs; cp_time and cp_dose indicate treatment duration (24, 48, 72 hours) and dose (high or low).
 - train_drug.csv - anonymous drug_id.
 - train_targets_scored.csv - The binary MoA targets that are scored.

## Preprocessing Steps

In capstone_preprocess.ipynb, features are one-hot encoded, log-normalised and transformed into image matrices of dimensions (300, 300, 3) using the t-SNE dimensionality reduction algorithm (DeepInsight Transformer). dataset is split into train, validation and test, then converted into TFRecord shards and uploaded along with one-hot encoded labels to Google Cloud storage bucket.

 - https://www.nature.com/articles/s41598-019-47765-6#Sec11

## Models

The EfficientNetB3 CNN algorithm is used to classify t-SNE transformed image matrices into one or more MoA targets. This model was selected based on high performance in this task in Kaggle competitions:

 - https://www.kaggle.com/code/cdeotte/3rd-place-public-lb-1805#Build-Ensemble

Two pretrained EfficientNetB3 algorithms are evaluated, one trained on the imagenet dataset, and another on the imagenet dataset using 'noisy student' training.

Noisy student training is a semi-supervised learning approach. It extends the idea of self-training and distillation with the use of equal-or-larger student models and noise added to the student during learning. It has three main steps:

1. train a teacher model on labeled images.
2. use the teacher to generate pseudo labels on unlabeled images.
3. train a student model on the combination of labeled images and pseudo labeled images.

Base models weights were set to trainable, as the dataset used in this project is very different from the imagenet images. A slow learning rate was used and Google Cloud TPU was utilised to accelerate learning. 

## Notebooks

 - capstone_EDA.ipynb
 - capstone_preprocess.ipynb
 - capstone_train_efficientnetb3_TPU.ipynb
 - capstone_train_efficientnetB3_noisystudent_TPU.ipynb
